In [11]:
from datetime import datetime
import psycopg2
import pandas as pd
import dotenv
import os
import requests
from requests.auth import HTTPBasicAuth

dotenv.load_dotenv()

True

In [2]:
connection = psycopg2.connect(host="localhost", database="github", user="yudai",password="yudaiap0913")

In [24]:
def get_events(user):
    auth = HTTPBasicAuth(
        os.environ.get("GITHUB_USERNAME"), os.environ.get("GITHUB_TOKEN")
    )
    data = requests.get(f"https://api.github.com/users/{user}/events?per_page=500", auth=auth)
    data = data.json()

    return data

In [7]:
events = get_repos("oracle")

for e in events:
    if e["type"] != "PullRequestEvent" and e["type"] != "IssuesEvent":
        continue
    
    event_id = e["id"]
    username = e["actor"]["login"]
    event_type = e["type"]
    repo_name = e["repo"]["name"]
    if event_type == "PullRequestEvent":
        commits = e["payload"]["pull_request"]["commits"]
        additions = e["payload"]["pull_request"]["additions"]
        deletions = e["payload"]["pull_request"]["deletions"]
        file_changes = e["payload"]["pull_request"]["changed_files"]
        number = e["payload"]["number"]
    else:    
        number = e["payload"]["issue"]["number"]
        commits = 0
        additions = 0
        deletions = 0
        file_changes = 0
        
    created_at = e["created_at"]
    
    print(f"ID: {event_id}")
    print(f"username: {username}")
    print(f"type: {event_type}")
    print(f"commits: {commits}")
    print(f"addition: {additions}")
    print(f"deletions: {deletions}")
    print(f"file changes: {file_changes}")
    print(f"number: {number}")
    print(f"created_at: {created_at}")
    print("")

ID: 13634735892
username: kishore-oracle
type: IssuesEvent
commits: 0
addition: 0
deletions: 0
file changes: 0
number: 255
created_at: 2020-09-25T00:45:28Z

ID: 13634735161
username: kishore-oracle
type: IssuesEvent
commits: 0
addition: 0
deletions: 0
file changes: 0
number: 19
created_at: 2020-09-25T00:45:20Z

ID: 13634734281
username: kishore-oracle
type: IssuesEvent
commits: 0
addition: 0
deletions: 0
file changes: 0
number: 54
created_at: 2020-09-25T00:45:10Z

ID: 13634733175
username: kishore-oracle
type: IssuesEvent
commits: 0
addition: 0
deletions: 0
file changes: 0
number: 24
created_at: 2020-09-25T00:44:59Z

ID: 13634732003
username: kishore-oracle
type: IssuesEvent
commits: 0
addition: 0
deletions: 0
file changes: 0
number: 289
created_at: 2020-09-25T00:44:45Z

ID: 13634688170
username: mark-au
type: IssuesEvent
commits: 0
addition: 0
deletions: 0
file changes: 0
number: 1533
created_at: 2020-09-25T00:36:18Z

ID: 13634652721
username: nalsaber
type: PullRequestEvent
commits: 

In [8]:
users = [
    "Snailclimb", "hmkcode", "HelloWorld521", "gaopu", "singgel", "kdn251", "macrozheng", "PhilJay",
    "crossoverJie", "ityouknow", "xkcoding", "hollischuang", "judasn", "looly", "vermarajat22255", 
    "kenshin579", "microlong666", "sangwoo24", "HagenGaryP", "matihost", "BehnamKhazael"
]

def connect(user: str, dbname: str, password: str):
    return psycopg2.connect(
        " user=" + user + " dbname=" + dbname + " password=" + password
    )


In [17]:
dbname = "github"
password = "yudaiap0913"

for user in users:
    events = get_events(user)
    
    for e in events:
        if e["type"] != "PullRequestEvent" and e["type"] != "IssuesEvent":
            continue

        event_id = e["id"]
        username = e["actor"]["login"]
        event_type = e["type"]
        repo_name = e["repo"]["name"]
        if event_type == "PullRequestEvent":
            commits = e["payload"]["pull_request"]["commits"]
            additions = e["payload"]["pull_request"]["additions"]
            deletions = e["payload"]["pull_request"]["deletions"]
            file_changes = e["payload"]["pull_request"]["changed_files"]
            number = e["payload"]["number"]
        else:    
            number = e["payload"]["issue"]["number"]
            commits = 0
            additions = 0
            deletions = 0
            file_changes = 0
        
        created_at = datetime.strptime(e["created_at"],  "%Y-%m-%dT%H:%M:%SZ")
            
        query = (
            "INSERT INTO EventTable VALUES "
            + f"({event_id}, '{username}', '{event_type}', '{repo_name}', {commits}, {additions}, "
            + f"{deletions}, {file_changes}, {number}, '{created_at}');"
        )

        with connect("yudai", dbname, password) as conn:
            with conn.cursor() as cur:
                try:
                    cur.execute(query)
                    conn.commit()
                except: 
                    print(f"{event_id} is passed")
                    continue
        
    print("registered ", user)

registered  Snailclimb
registered  hmkcode
registered  HelloWorld521
registered  gaopu
registered  singgel
registered  kdn251
registered  macrozheng
registered  PhilJay
registered  crossoverJie
registered  ityouknow
registered  xkcoding
registered  hollischuang
registered  judasn
registered  looly
registered  vermarajat22255
registered  kenshin579
registered  microlong666
registered  sangwoo24
registered  HagenGaryP
registered  matihost
registered  BehnamKhazael


In [41]:
query = "SELECT * FROM EventTable;"

df = pd.read_sql(query, connection)
df.head()

,event_id,username,event_type,repository_name,commits,additions,deletions,changed_files,event_number,created_at
0,13609911999,Snailclimb,IssuesEvent,Snailclimb/awesome-java,0,0,0,0,9,2020-09-23 06:24:21
1,13561876234,Snailclimb,IssuesEvent,Snailclimb/awesome-java,0,0,0,0,11,2020-09-18 05:41:53
2,13548014495,Snailclimb,PullRequestEvent,Snailclimb/JavaGuide,1,1,1,1,934,2020-09-17 05:39:27
3,13547945496,Snailclimb,PullRequestEvent,Snailclimb/JavaGuide,1,1,1,1,933,2020-09-17 05:29:47
4,13546124027,Snailclimb,PullRequestEvent,Snailclimb/guide-rpc-framework,2,92,23,5,14,2020-09-17 00:24:03


In [20]:
len(df)

199

In [23]:
len(df[df.event_type == "PullRequestEvent"])

101

In [26]:
def get_repos(user, repo_name):
    auth = HTTPBasicAuth(
        os.environ.get("GITHUB_USERNAME"), os.environ.get("GITHUB_TOKEN")
    )
    data = requests.get(f"https://api.github.com/repos/{user}/{repo_name}", auth=auth)
    data = data.json()

    return data

In [29]:
user, repo = df["repository_name"][0].split("/")
get_repos(user, repo)

{'id': 226310377,
 'node_id': 'MDEwOlJlcG9zaXRvcnkyMjYzMTAzNzc=',
 'name': 'awesome-java',
 'full_name': 'Snailclimb/awesome-java',
 'private': False,
 'owner': {'login': 'Snailclimb',
  'id': 29880145,
  'node_id': 'MDQ6VXNlcjI5ODgwMTQ1',
  'avatar_url': 'https://avatars0.githubusercontent.com/u/29880145?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/Snailclimb',
  'html_url': 'https://github.com/Snailclimb',
  'followers_url': 'https://api.github.com/users/Snailclimb/followers',
  'following_url': 'https://api.github.com/users/Snailclimb/following{/other_user}',
  'gists_url': 'https://api.github.com/users/Snailclimb/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/Snailclimb/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/Snailclimb/subscriptions',
  'organizations_url': 'https://api.github.com/users/Snailclimb/orgs',
  'repos_url': 'https://api.github.com/users/Snailclimb/repos',
  'events_url': 'https://api.github.co

In [42]:
for repo in df["repository_name"].unique():
    username, repo_name = repo.split("/")
    
    repo_info = get_repos(username, repo_name)
    star = repo_info["stargazers_count"]
    fork = repo_info["forks"]
    language = repo_info["language"]
    size = repo_info["size"]
    created_at = datetime.strptime(repo_info["created_at"], "%Y-%m-%dT%H:%M:%SZ")
    
    if language != None:
        query = (f"INSERT INTO RepoTable VALUES ('{repo_name}', '{username}', {star}, {fork}, '{language}', "
                 + f"{size}, '{created_at}',  null);")
    else:
        query = (f"INSERT INTO RepoTable VALUES ('{repo_name}', '{username}', {star}, {fork}, null, "
                 + f"{size}, '{created_at}',  null);")
    
    with connect("yudai", dbname, password) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

In [46]:
query = "SELECT * FROM RepoTable;"

df_repo = pd.read_sql(query, connection)
df_repo.language.unique()

array([None, 'Java', 'Vue', 'JavaScript', 'Python', 'Less', 'HTML'],
      dtype=object)